<a href="https://colab.research.google.com/github/KESAVE8604-droid/kesaverdhen-hybrid-movie-recommender/blob/main/Phase6_WebApp_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/MovieRecommender/


Mounted at /content/drive
/content/drive/MyDrive/MovieRecommender


In [2]:
!pip install streamlit joblib scikit-learn pandas numpy scikit-surprise --quiet
!pip install --force-reinstall numpy==1.26.4 scikit-surprise==1.1.4 --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 64.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.4/308.4 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.7/35.7 MB 23.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; pyt

In [3]:
%%writefile /content/drive/MyDrive/MovieRecommender/app.py
# 🎬 Hybrid Movie Recommender System — Streamlit App
import streamlit as st
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics.pairwise import cosine_similarity

# -------------------------------
# 🎬 Load Pre-trained Models
# -------------------------------
@st.cache_resource
def load_models():
    tfidf = joblib.load("tfidf_vectorizer.pkl")
    tfidf_matrix = joblib.load("tfidf_matrix.pkl")
    cosine_sim = joblib.load("cosine_similarity.pkl")
    cf_model = joblib.load("collab_model.pkl")
    movies = pd.read_csv("movielens_cleaned.csv").drop_duplicates(subset=['MovieID'])
    return tfidf, tfidf_matrix, cosine_sim, cf_model, movies

tfidf, tfidf_matrix, cosine_sim, cf_model, movies = load_models()

st.set_page_config(page_title="🎥 Movie Recommender", layout="wide")

# -------------------------------
# 🧩 Utility Functions
# -------------------------------
indices = pd.Series(movies.index, index=movies['Clean_Title'].str.lower()).drop_duplicates()

def content_recommend(movie_title, top_n=10):
    title = movie_title.lower().strip()
    if title not in indices:
        return pd.DataFrame(columns=['Clean_Title', 'Genres', 'Year'])
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:top_n+1]
    movie_indices = [i[0] for i in sim_scores]
    return movies.iloc[movie_indices][['Clean_Title', 'Genres', 'Year']].reset_index(drop=True)

def hybrid_recommend(user_id, liked_movie, alpha=0.7, top_n=10):
    content_df = content_recommend(liked_movie, top_n)
    if content_df.empty:
        return content_df
    hybrid_scores = []
    for mid in content_df.index:
        movie_id = movies.iloc[mid]['MovieID']
        try:
            cf_score = cf_model.predict(user_id, movie_id).est
        except:
            cf_score = 3.0
        content_score = cosine_sim[indices[liked_movie.lower()]][mid]
        hybrid = alpha * cf_score + (1 - alpha) * content_score * 5
        hybrid_scores.append(hybrid)
    content_df["Hybrid_Score"] = hybrid_scores
    return content_df.sort_values(by="Hybrid_Score", ascending=False).head(top_n)

# -------------------------------
# 🖥️ Streamlit UI
# -------------------------------
st.title("🎬 Hybrid Movie Recommender System")
st.markdown("##### Powered by **Content + Collaborative Filtering**")

col1, col2 = st.columns([2,1])
with col1:
    movie_name = st.selectbox(
        "🎞️ Select your favorite movie:",
        sorted(movies["Clean_Title"].unique())
    )
with col2:
    user_id = st.number_input("👤 Enter User ID:", min_value=1, value=1, step=1)

alpha = st.slider("⚖️ Weight α (CF vs Content)", 0.0, 1.0, 0.7, 0.1)

if st.button("🔍 Get Recommendations"):
    with st.spinner("Generating recommendations..."):
        recs = hybrid_recommend(user_id, movie_name, alpha)
    if not recs.empty:
        st.success(f"Top {len(recs)} Recommendations for **{movie_name}** (User {user_id})")
        st.dataframe(recs.style.background_gradient(cmap="Blues"))
    else:
        st.warning("Movie not found. Please try another title.")

st.caption("📚 Developed by Kesaverdhen A — 22BDS0022")


Writing /content/drive/MyDrive/MovieRecommender/app.py


In [ ]:
!streamlit run /content/drive/MyDrive/MovieRecommender/app.py --server.enableCORS false --server.enableXsrfProtection false --server.port 8501





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://35.236.183.50:8501

